In [ ]:
# I want to fact-check the February 2021 visualization from the Heritage foundation, 
# on their website with almost no commentary,
# https://datavisualizations.heritage.org/public-health/one-percent-of-us-counties-account-for-bulk-of-covid-19-deaths/
# "Through January 31, there have been 433,401 deaths in the U.S. from COVID-19, but a significant 
#  proportion of those have occurred in just a handful of counties. In fact, the 30 counties with 
#  the most deaths represent 1 percent of all counties, 21 percent of the total U.S. population, 
#  and 28 percent of all U.S. deaths."

In [ ]:
# The CDC has an API to serve county-level cases and deaths data:
# https://data.cdc.gov/NCHS/Provisional-COVID-19-Deaths-by-County-and-Race-and/k8wy-p9cg

# Initial tests suggest, though, that the data does not reach back to January 2021.
# Update 202

In [ ]:
# There is a huge, updated database on the NY Times github page:
# https://github.com/nytimes/covid-19-data 

# That contains a 75-Mb file us-counties.csv.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
counties = pd.read_csv("../data/us-counties.csv")

In [ ]:
counties.head()

In [ ]:
counties.sample(5)

In [ ]:
# Let us grab just the slice of deaths representing Cook Co:
cookco = counties.loc[np.where( counties.fips == 17031)]

In [ ]:
len(cookco)

In [ ]:
# 600 data points.. less than two years.. check.
plt.plot(cookco.date, cookco.deaths)

In [ ]:
# That took a long time and seems like it was plotting nominal time data.   
# Search engine, "pandas to date format," please?
# https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
# Thanks.

In [ ]:
d = pd.to_datetime(counties["date"])


In [ ]:
d


In [ ]:
counties["date"] = d
# Note, I'm overwriting the data by reformatting it here.

In [ ]:
cookco = counties.loc[np.where( counties.fips == 17031)]

In [ ]:
plt.plot(cookco.date, cookco.deaths)

In [ ]:
# Now everyone stare at this for a minute.  What is this the graph of ?

In [ ]:
#  A:  This is the cumulative sum of a bimodal distribution;
# There were two waves of high death rate separated by 
# periods of quiet. 

In [ ]:
# Let us plot per-day deaths:

In [ ]:
plt.plot(cookco.date[1:], np.diff(cookco.deaths))

In [ ]:
# It's a little noisy.  Differences between points 7 
# days apart will automatically average over 7-day intervals:

In [ ]:
smooth = cookco["deaths"] - cookco["deaths"].shift(7)

In [ ]:
cookco.deaths.head()

In [ ]:
cookco.deaths.shift(7).head(15)

In [ ]:
plt.plot(cookco.date, smooth)

In [ ]:
# This is overplotting.  (Is overplotting bad?)

In [ ]:
plt.plot(cookco.date[::7], smooth[::7])

In [ ]:
# Note a few things.
# 1. The scale changed, we're plotting weekly deaths now.
# 2. There are 7 data points per week, and each data point represents
# the preceding 7 days reporting--so the area under the graph is 7x 
# the true death rate.  This isn't exactly deceptive, but I don't want
# to fool myself either.

In [ ]:
# There is an initial wave of deaths in spring 2020, and a 
# second wave of roughly equal intensity November-March 2021.

In [ ]:
cookco.deaths

In [ ]:
len(counties)

In [ ]:
# And there were 650 days, so about this many countes
len(counties)/650

In [ ]:
# Take subset of the data at one point in time for comparison to Heritage
febdata = counties.loc[np.where(counties.date =="2021-01-31")]

In [ ]:
len(febdata)

In [ ]:
# If I encounter a TypeError at this stage, it is becuase I can't compare an ISO8601 
# date string to an np.datetime64-typed object.

In [ ]:
febdata = counties.loc[np.where(counties.date ==np.datetime64("2021-01-31"))]

In [ ]:
len(febdata)

In [ ]:
febdata.head()

In [ ]:
febdata.deaths.sum()

In [ ]:
# This is more than 433,401 deaths quoted on the Heratige site, but 
# we are clearly in the right ballpark here; the 8000 deaths might be 
# differences in the territorial scope of their query or the data may
# have been revised upward by 1.7%.  Onward!

In [ ]:
plt.hist(febdata.deaths, bins=100)

In [ ]:
# Ack.  That's not very expressive, is it?
# In vanilla matplotlib, you have to draw your own bins if you want a nice log-histogram.

# https://stackoverflow.com/questions/47850202/plotting-a-histogram-on-a-log-scale-with-matplotlib
    
plt.hist(np.log(febdata.deaths) / np.log(10), bins=100)

In [ ]:
# This is a "can't take logarithm of zero" error.
# I can fix (for now) by adding one one-hundredth
# and remembering that counties with .01 deaths 
# reported zero or NaN deaths in the database.
plt.hist(np.log(febdata.deaths + .01) / np.log(10), bins=100)


In [ ]:
# So I have to make my own logarithmicly spaced histogram bins.
bins = np.exp(np.arange(-1,4,.05))

In [ ]:
plt.hist(febdata.deaths , bins=bins )
plt.xscale("log")

In [ ]:
# What happened to my monomodal distribution?
# Domain is wrong.. only 10^(-0.5) to 10^2

In [ ]:
# Try coarser bins?
bins = np.exp(np.arange(-1,4,.1))
plt.hist(febdata.deaths , bins=bins )
plt.xscale("log")

In [ ]:
# That's still not right.  Am I using arange wrong?

In [ ]:
bins

In [ ]:
# That's not the right range.  I wanted 10^(-1) to 10^4.
# I wanted to use logs base 10, but forgot that the 
# inverse of log base 10 is not np.exp, it's np.power(10,x)

In [ ]:
bins = np.power(10, np.arange(-1,4,.1) )
plt.hist(febdata.deaths , bins=bins )
plt.xscale("log")
plt.xlabel("per-county deaths")

In [ ]:
# And it looks like now we're silently ignoring 
# the counties with no deaths becuase they're offscale:

In [ ]:
bins = np.power(10, np.arange(-1,4,.1) )
plt.hist(febdata.deaths+.1 , bins=bins )
plt.xscale("log")
plt.xlabel("per-county deaths")

In [ ]:
# This log-histogram really robs me of any sense of what the 
# distribution is actually like.
# Can we plot deaths by county's death rank?
deathsbycounty = febdata.sort_values("deaths")

In [ ]:
plt.plot(np.arange(len(deathsbycounty)), deathsbycounty.deaths)
plt.xlabel("County death rank")
plt.ylabel("Per-county cumulative COVID deaths, Feb 2021")

In [ ]:
# Rrr.  This still does not make me think I understand.  
# This is not a very expressive graph--one county has 27000 
# deaths, there are 3100 counties; I can tell little else,
# least of all the total body count.


In [ ]:
# Wait a sec, one county with 27,000 deaths, does that make sense?
# The largest county on Heratige's page was LA County with only 17,000.

In [ ]:
counties.sort_values("deaths")

In [ ]:
# These NaN values for deaths are getting in my way, I'm going to reset them
# to zero now.
counties[np.isnan(counties.deaths)] =0

In [ ]:
# And I better update the data frames representing convenient subsets too
febdata = counties.loc[np.where(counties.date ==np.datetime64("2021-01-31"))]
cookco = counties.loc[np.where( counties.fips == 17031)]
deathsbycounty = febdata.sort_values("deaths")

In [ ]:
counties.sort_values("deaths")

In [ ]:
# Now I can see the deathmost counties.  NYC has its own codepoint.
# Check.. is NYC counted twice ?

In [ ]:
"Queens" in counties.county.values

In [ ]:
"Kings" in counties.county.values

In [ ]:
counties.loc[np.where(counties.county == "Kings")]

In [ ]:
counties.loc[np.where(counties.county == "Kings")]["state"].value_counts()

In [ ]:
# Now I am convinced that we are not double-counting NYC.
# Our data source does not give us borough-level detail tho.

In [ ]:
# The legend says "cumulative"
# meaning cumulative by time.  Let's try cumulative by county. 

In [ ]:
plt.plot(np.arange(len(deathsbycounty)), np.cumsum(deathsbycounty.deaths))
plt.xlabel("County death rank")
plt.ylabel("Per-county cumulative COVID deaths, Feb 2021")

In [ ]:
plt.plot(np.arange(len(deathsbycounty)), np.cumsum(deathsbycounty.deaths.sort_values(ascending=False)))
plt.xlabel("County death rank")
plt.ylabel("Per-county cumulative COVID deaths, Feb 2021")

In [ ]:
# Let us plot these side-by-side
plt.figure(figsize=(14,6))
plt.subplot(121)
plt.plot(np.arange(len(deathsbycounty)), np.cumsum(deathsbycounty.deaths.sort_values(ascending=True)))
plt.xlabel("County size rank")
plt.ylabel("Per-county cumulative COVID deaths, Feb 2021")
plt.subplot(122)
plt.plot(np.arange(len(deathsbycounty)), np.cumsum(deathsbycounty.deaths.sort_values(ascending=False)))
plt.xlabel("County size rank")
plt.ylabel("Per-county cumulative COVID deaths, Feb 2021")

In [ ]:
# This is getting closer.  Clearly more than half of deaths are in a 
# small fracton of the counties; on linear scales, these graphs
# are the same, but counting from largest to smallest vs. smallest
# to largest matters if you want to transform either axis by,
# for instance, logarithmic scaling.

In [ ]:
# top 30 counties, fraction of deaths?
deathsbycounty.head()

In [ ]:
deathsbycounty[-30:]

In [ ]:
np.sum(deathsbycounty.deaths[-30:]) / np.sum(deathsbycounty.deaths)

In [ ]:
np.sum(deathsbycounty.deaths[-30:])

In [ ]:
counties.sort_values("deaths")[-10:]

In [ ]:
deathsbycounty[-10:]

In [ ]:
# Now we need to go fetch population by county 
# This table purports to assemble 2020 census data in a usable table
# https://github.com/nytimes/covid-19-data/issues/180
population = pd.read_json("https://raw.githubusercontent.com/Zoooook/CoronavirusTimelapse/master/static/population.json")

In [ ]:
population.head()

In [ ]:
population.sample(5)

In [ ]:
population.population.sum()

In [ ]:
# 326 million.. this is more recent than the 2010 census then.

In [ ]:
countiesbypop = population.population.sort_values(ascending=False)

In [ ]:
plt.plot(np.arange(len(countiesbypop)), countiesbypop)

In [ ]:
plt.plot(np.arange(len(countiesbypop)), np.cumsum(countiesbypop))
plt.xlabel("County population rank")
plt.ylabel("Cumulative population")

In [ ]:
deathsbycountyrev = febdata.deaths.sort_values(ascending=False)

In [ ]:
plt.plot(np.arange(len(countiesbypop)), np.cumsum(countiesbypop))
plt.plot(np.arange(len(deathsbycountyrev)), np.cumsum(deathsbycountyrev))

In [ ]:
# -1 point for not being expressive.  326 million and 400k don't
# fit on the same graph.  Log transform?

In [ ]:
plt.plot(np.arange(len(countiesbypop)), np.cumsum(countiesbypop))
plt.plot(np.arange(len(deathsbycountyrev)), np.cumsum(deathsbycountyrev))
plt.yscale("log")

In [ ]:
# I am being intellecutally dishonest here, watch out. 
# Because I've not joined the two tables on the fips county id yet,
# I am presenting two sorted series of counties 
# that have been sorted by different columns
# as if they are the same.  One is sorted by population;
# one is sorted by deaths.  But each row has its own identity, 
# and there is a one-to-one relationship betwen population and
# deaths that I have NOT yet shown.

In [ ]:
population.head(1)

In [ ]:
febdata.head(1)

In [ ]:
# I have to note that the deaths data encodes fips as a 
# floating point number, while the population data enocdes 
# it as an integer.  This is probably enough to prevent me
# from joining it.  


In [ ]:
febdata.fips.dtype

In [ ]:
# can I recast it as an integer?
febdata.fips.astype("int64")

In [ ]:
# There are some bad values in there.

In [ ]:
# Can I ignore errors?
fips= febdata.fips.astype("int64", errors="ignore")

In [ ]:
fips.head()

In [ ]:
fips.dtype

In [ ]:
# No.  If I ignore errors the result isn't in an integer data type. 
fips= febdata.fips.astype(np.int64, errors="ignore")
fips.dtype

In [ ]:
# How many bad values are there?

In [ ]:
np.isinf(fips).sum()

In [ ]:
np.isnan(fips).sum()

In [ ]:
# Let's look at them:
counties.loc[np.isnan(counties.fips)]

In [ ]:
# Ugh oh.  These are deaths I probably want to count, despite
# the fact that I won't be able to join them with population.  
# Only 29 bad fips fields in febdata:

In [ ]:
febdata.loc[np.isnan(febdata.fips)]

In [ ]:
febdata.loc[np.isnan(febdata.fips)].deaths.sum()

In [ ]:
# Amounting to 31,000 deaths.

In [ ]:
27138.0+1829.0

In [ ]:
febdata.fips.value_counts()

In [ ]:
# 31,000 deaths, almost all of which are from NYC and Puerto Rico.
# I'm willing to treat them as special cases.

In [ ]:
# Some trial-and-error suggests that I must fix NaN's first,
# then change the type to int.
febdata.loc[np.isnan(febdata.fips),"fips"] = 999999
fips = febdata.fips.astype(np.int64)
fips.dtype

In [ ]:
fips.head()

In [ ]:
np.isnan(febdata.fips).sum()

In [ ]:
febdata["fips"] = febdata.fips.astype(np.int64)

In [ ]:
np.isnan(counties.fips).sum()

In [ ]:
counties.head(1)

In [ ]:
# That seemed to work for febdata, apply to counties
counties.loc[np.isnan(counties.fips),"fips"] = 999999

In [ ]:
np.isnan(counties.fips).sum()

In [ ]:
counties["fips"] = counties.fips.astype(np.int64)
counties.fips.dtype

In [ ]:
febdata.loc[np.isnan(febdata.fips)]

In [ ]:
population.loc[population.region == "Puerto Rico"]

In [ ]:
population.loc[population.subregion == "New York City"]

In [ ]:
febdata[febdata.county == "New York City"]

In [ ]:
febdata.loc[febdata.county == "New York City","fips"] = 36998

In [ ]:
# and I should change this for PR and counties too
febdata.loc[febdata.state == "Puerto Rico","fips"] = 72999
counties.loc[counties.county == "New York City","fips"] = 36998
counties.loc[counties.state == "Puerto Rico","fips"] = 72999


In [ ]:
febdata.head(1)

In [ ]:
febdata.fips.dtype

In [ ]:
population.head(1)

In [ ]:
# duplicate the field with a different name for convenience
population["fips"]= population.us_county_fips

In [ ]:
deathswithpop= counties.join(population, "fips")

In [ ]:
# No suffix specified.. ok, here's a suffix
deathswithpop= counties.join(population, "fips", rsuffix="_pop")

In [ ]:
deathswithpop.head(2)

In [ ]:
# Well maybe it never joins Snohomish county for some reason...
deathswithpop.loc[::1000]

In [ ]:
#  Most of these are not joined
plt.plot(deathswithpop.deaths, deathswithpop.population)

In [ ]:
# Turn off the lines...
plt.plot(deathswithpop.population, deathswithpop.deaths,'.')

In [ ]:
# loglog plot..
plt.plot(deathswithpop.population, deathswithpop.deaths,'.')
plt.xscale("log"); plt.yscale("log")

In [ ]:
# This does not look right. County population should not have
# many values for well-separated points in the middle of the domain.
# Unless I'm plotting all the time points on top of each other.  D'oh.

febdata = deathswithpop.loc[np.where(deathswithpop.date ==np.datetime64("2021-01-31"))]

In [ ]:
# Check that numbers add up!
febdata.population.sum()

In [ ]:
febdata.deaths.sum()

In [ ]:
# All the deaths, but almost none of the population.  7M / 326M 

In [ ]:
# Let's spot-check Wyandotte County, KS.  Why did it not join?
population.loc[np.where(population.subregion=="Wyandotte")]

In [ ]:
counties.loc[ np.where(counties.county == "Wyandotte")]

In [ ]:
deathswithpop= counties.join(population, "fips", rsuffix="_pop")
deathswithpop[::10000]

In [ ]:
# Not joined
deathswithpop.deaths.sum()

In [ ]:
deathswithpop.head()

In [ ]:
feb = deathswithpop.loc[np.where(deathswithpop.date ==np.datetime64("2021-01-31"))]


In [ ]:
len(feb)

In [ ]:
feb.population.sum()

In [ ]:
len(feb[feb.population>0])

In [ ]:
# A little bit of a puzzle why only 94 counties out of 3169 joined.
# Search engine...
# https://stackoverflow.com/questions/10114399/pandas-simple-join-not-working
# Use pd.DataFrame.merge; pd.join pays too much attention to indexes, and we often
# want to join on data columns that aren't suitable indexes.

deathswithpop= counties.merge(population, on="fips")
feb = deathswithpop.loc[np.where(deathswithpop.date ==np.datetime64("2021-01-31"))]


In [ ]:
feb.population.sum()

In [ ]:
feb.deaths.sum()

In [ ]:
# So we lost a million population and two thousand deaths, but most rows merged!

In [ ]:
len(feb.loc[feb.population>0])

In [ ]:
feb.head()

In [ ]:
feb[::100]

In [ ]:
# I am now satisfied that my table is joined.

In [ ]:
feb["percapitadeaths"] = feb.deaths / feb.population

In [ ]:
plt.scatter(feb.population, feb.deaths)
plt.xlabel("County Population")
plt.ylabel("County Deaths")

In [ ]:
plt.loglog(feb.population, feb.deaths,'.')
plt.xlabel("County Population")
plt.ylabel("County Deaths")

In [ ]:
# What do you see in this graph?


In [ ]:
# Just for kicks... histogram of per capita death rate?
plt.hist (feb.percapitadeaths, bins=100)
plt.xlabel("Per capita death rate")

In [ ]:
# This looks believable, but this is not the graph on which 
# we will find the truth. 
# WE ARE COUNTING THE WRONG THINGS, as usual.
# Imagine the headline "50% of counties report death rates above
feb.percapitadeaths.median()
# .00134  ( one death per 742 population).  This generalizes how?
# The counties do not all have the same weights.

In [ ]:
# To get at the Heritige Foundation's implicit question -- how can we show
# the difference between death toll in large and small (population, before-the-fact)
# counties, I suggest plotting cumulative population (small to large) against 
# cumulative deaths.

In [ ]:
plt.plot(feb.population.cumsum())

In [ ]:
# Och, forgot to sort by population.
plt.plot( feb.population.sort_values(ascending=True).cumsum())

In [ ]:
# Uh, what?   
feb.population.sort_values()

In [ ]:
# plt.plot wants to plot using the index.  ok.  I can plot against rank with np.arange(len(feb.population)

In [ ]:
# This gives me cumulative population from smallest to biggest... 
plt.subplot(121);
plt.plot(np.arange(len(feb.population)), feb.population.sort_values(ascending=True).cumsum())
# And biggest to smallest... 

plt.subplot(122)
plt.plot(np.arange(len(feb.population)), feb.population.sort_values(ascending=False).cumsum())

In [ ]:
# Consider (as the Heratige people did) this as a data transformation: 
# this exaggerates small counties by changing the x-axis.

In [ ]:
# but I'm using sort_values on feb.population (a series); I need the whole dataframe for 
# the next step. 
plt.subplot(121);
plt.plot(np.arange(len(feb.population)), feb.sort_values("population", ascending=True).population.cumsum())
plt.subplot(122)
plt.plot(np.arange(len(feb.population)), feb.sort_values("population", ascending=False).population.cumsum())

In [ ]:
# This gives me the same thing, but also gives me access to deaths in the same order.
# but I'm using sort_values on feb.population (a series); I need the whole dataframe for 
# the next step. 
plt.subplot(121);
plt.plot(np.arange(len(feb.population)), feb.sort_values("population", ascending=True).population.cumsum())
plt.plot(np.arange(len(feb.population)), feb.sort_values("population", ascending=True).deaths.cumsum())
plt.subplot(122)
plt.plot(np.arange(len(feb.population)), feb.sort_values("population", ascending=False).population.cumsum())
plt.plot(np.arange(len(feb.population)), feb.sort_values("population", ascending=False).deaths.cumsum())


In [ ]:
# Make it logarithmic just to make sure we have what we expect
plt.subplot(121);
plt.plot(np.arange(len(feb.population)), feb.sort_values("population", ascending=True).population.cumsum())
plt.plot(np.arange(len(feb.population)), feb.sort_values("population", ascending=True).deaths.cumsum())
plt.ylabel("Cumulative population")
plt.xlabel("County population rank (small to large)")
plt.yscale("log")
plt.subplot(122)
plt.plot(np.arange(len(feb.population)), feb.sort_values("population", ascending=False).population.cumsum())
plt.plot(np.arange(len(feb.population)), feb.sort_values("population", ascending=False).deaths.cumsum())
plt.yscale("log")
#plt.ylabel("Cumulative population")
plt.xlabel("County population rank (large to small)")


In [ ]:
# And now for the finish, plot cumulative population vs. cumulative deaths.
# Before we plot it, we expect it will go from 0,0 to 326M, 411k.
plt.figure(figsize=(16,8))
plt.subplot(121);
plt.plot( feb.sort_values("population", ascending=True).population.cumsum(), feb.sort_values("population", ascending=True).deaths.cumsum())
plt.xlabel("cumulative of population, smallest to largest counties (326M)")
plt.ylabel("cumulative of COVID deaths n = 411k")

plt.subplot(122)
plt.plot( feb.sort_values("population", ascending=False).population.cumsum(), feb.sort_values("population", ascending=False).deaths.cumsum())
plt.xlabel("cumulative of population, largest to smallest counties (326M)")
plt.ylabel("cumulative COVID deaths (Feb 2021) n = 411k")


In [ ]:
# ?!?!?!

In [ ]:
# I'm using calculus to make my data smoother.
# Yeah, there's a tiny kink on the left hand corner, but we kind of already knew that
# King County, WA, and NYC were the hardest hit by the disease in Q1 2020.  

In [ ]:
# add line at constant-fraction-population-deaths

plt.figure(figsize=(16,8))
plt.subplot(121);
plt.plot( feb.sort_values("population", ascending=True).population.cumsum() / feb.population.sum(), feb.sort_values("population", ascending=True).deaths.cumsum()/feb.deaths.sum())
plt.xlabel("fraction of population, smallest to largest counties (326M)")
plt.ylabel("fraction of COVID deaths n = 411k")

plt.plot([0,1],[0,1])
plt.subplot(122)
plt.plot( feb.sort_values("population", ascending=False).population.cumsum() / feb.population.sum(), feb.sort_values("population", ascending=False).deaths.cumsum()/feb.deaths.sum())
plt.xlabel("fraction of population, largest to smallest counties (326M)")
plt.ylabel("fraction of COVID deaths n=411k")

plt.plot([0,1],[0,1])

In [ ]:
# This is an honest visualization, but 
# I was not expecting to see this.  

# All right, I was shocked that the effect size seems so small.

In [ ]:
# The effect is real, but seems smaller than we were led to believe.  (Hard to judge from graph)
# Note these two graphs have the same shape, just flipped up and down and left and right.
# On the right-hand graph, we see that large counties are in fact harder hit, 
# and that the third of the population in the smallest counties had lower-than-overall
# death rates.   

# Hmm.  How to make this deceptive?  

# A:  Sort by after-the-fact deaths by county (something you should never do)


In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(121);
plt.plot( feb.sort_values("deaths", ascending=True).population.cumsum() / feb.population.sum(), feb.sort_values("deaths", ascending=True).deaths.cumsum()/feb.deaths.sum())
plt.xlabel("fraction of population, fewest to most death counties (326M)")
plt.ylabel("fraction of COVID deaths n = 411k")

plt.plot([0,1],[0,1])
plt.subplot(122)
plt.plot( feb.sort_values("deaths", ascending=False).population.cumsum() / feb.population.sum(), feb.sort_values("deaths", ascending=False).deaths.cumsum()/feb.deaths.sum())
plt.xlabel("fraction of population, most deaths to least deaths counties (326M)")
plt.ylabel("fraction of COVID deaths n=411k")

plt.plot([0,1],[0,1])

In [ ]:
# This is not as dramatic as I was expecting. 
# Let's see if it matches.
#  65 counties most deaths - 31.9% US population 40.3% of deaths

heritage = pd.DataFrame( {
"fracdeaths": feb.sort_values("deaths", ascending=False).deaths.cumsum() / feb.deaths.sum(),
"fracpopulation": feb.sort_values("deaths", ascending=False).population.cumsum() / feb.population.sum()
})

In [ ]:
heritage.index = np.arange(len(heritage))

In [ ]:
heritage

In [ ]:
heritage.loc[64]

In [ ]:
heritage.loc[60]

In [ ]:
#  65 counties most deaths - 31.9% US population 40.3% of deaths
# This is a size of error that might be caused by consolidating 
# NYC's five boroughs.. 

# The difference at 40.3% of deaths is 0.5% of US population, 
# which is a difference of a county (or borough) of 1.6M population.
# This isn't far enough off to make me sound the alarm.  

# The deception was in sorting by the outcome of a random process.

In [ ]:
# Maybe I can plot the difference betwen fraction population and fraction of deaths?
plt.figure(figsize=(16,8))
plt.subplot(121);
plt.plot( feb.sort_values("deaths", ascending=True).population.cumsum() / feb.population.sum(),    feb.sort_values("deaths", ascending=True).deaths.cumsum()/feb.deaths.sum()- feb.sort_values("deaths", ascending=True).population.cumsum() / feb.population.sum())
plt.xlabel("fraction of population, least deaths to most death counties (326M)")
plt.ylabel("difference fraction of covid deaths to fraction of population ")

plt.plot([0,1],[0,0])
plt.subplot(122)
plt.plot( feb.sort_values("population", ascending=False).population.cumsum() / feb.population.sum(), feb.sort_values("population", ascending=False).deaths.cumsum()/feb.deaths.sum()- feb.sort_values("population", ascending=False).population.cumsum()/feb.population.sum( ))
plt.xlabel("fraction of population, most populous to least populous counties (326M)")
plt.ylabel("difference fraction of covid deaths to fraction of population")
plt.plot([0,1],[0,0])



In [ ]:
# And this relationship on the left was treated with a slider that
# ran from 0 to 5% of population, and with another slider for the n
# most-death counties that ran from 52% to 89%. 

In [ ]:
# Let us consider that histogram of per-county deaths..
# It was weighted such that each county got one vote. 
bins = np.power(10, np.arange(-1,4,.1) )
plt.hist(febdata.deaths , bins=bins )
plt.xscale("log")
plt.xlabel("per-county deaths")
plt.ylabel("Number of counties")

In [ ]:
# We can add weights to the histogram to count people
bins = np.power(10, np.arange(-1,4,.1) )
plt.hist(feb.deaths+.1 , bins=bins, weights=feb.population )
plt.xscale("log")

plt.xlabel("per-county deaths")
plt.ylabel("population in affected counties")

In [ ]:
# And plot them at the same time
plt.figure(figsize=(8,8))
plt.subplot(211)
bins = np.power(10, np.arange(-1,4,.1) )
plt.hist(feb.deaths+.1 , bins=bins, alpha=0.5 )
plt.xscale("log")
plt.xlabel("per-county deaths")
plt.ylabel("number of counties")
plt.subplot(212)
plt.hist(feb.deaths+.1 , bins=bins, weights=feb.population, alpha=0.5 )
plt.xscale("log")
plt.xlabel("per-county deaths")
plt.ylabel("affected population")



In [ ]:
# These don't look as similar as one might want them to be.
# The median county has 30 deaths but the median citizen 
# lives in a county with 300 deaths.  

# Counting by county is bad.

In [ ]:
#rates?
plt.figure(figsize=(8,12))
plt.subplot(211)
plt.hist(feb.percapitadeaths , alpha=0.5, bins=100 )
plt.xlabel("per-capita COVID deaths (by county)")
plt.subplot(212)
plt.hist(feb.percapitadeaths , weights=feb.population, alpha=0.5 , bins=100)
plt.xlabel("per capita COVID deaths (by population)")

In [ ]:
# The effect here is not as strong, or is harder to see, but 
# this is the graph whose mean tells you how bad it is on average.

# For kicks, let's calculate the center of mass of these two histograms..
print ("Average crude death rate, weighted by county   ", feb.percapitadeaths.mean())
print ("Average crude death rate, weigted by population", (feb.percapitadeaths * feb.population).sum() / feb.population.sum())

In [ ]:
# Couldn't have seen that difference just looking at ragged histograms

In [ ]:
# I feel inspired to make variable-width bar charts from this death data. 

In [ ]:
# First, let me define some dummy data, and make sure I can plot this like a set
# of boxes piled up in a mail room.
d = pd.DataFrame ( {
"x": [ 1,1,1,2,1,2,1,2,1,2],
"y": [ 1,1,1,2,1,2,4,3,4,3]
})

In [ ]:
plt.bar(d.x.cumsum(),d.y, width=d.x)

In [ ]:
# This does not look right; there are gaps between the boxes.  And the first box starts at 0.5, not at 0.
d.x.cumsum()

In [ ]:
# I want my x's to start at zero..

In [ ]:
# create a dataframe for the x offsets
xoff  = pd.DataFrame({"x":[0], "y":[0]}).append([d]).x.cumsum()[:-1]
xoff

In [ ]:
plt.bar(xoff, d.y, width=d.x)

In [ ]:
# This is still not right; the middle of the boxes is being aligned with the cumulative sum just below each box.
# The fix for this is plt.bar align="edge"

In [ ]:
plt.bar(xoff, d.y, width=d.x, align="edge")

In [ ]:
# so this is my recipe for packing boxes into the moving van:
# prepend data with 0
# take cumulative sum and chop off final value
# set width = values and align = "edge"


In [ ]:
# There was a way to prepend the data with 0 with .shift()... it throws away the last data point.
d

In [ ]:
d.shift(1)

In [ ]:
d.shift(1,fill_value=0)

In [ ]:
# Well that seems to do it.

In [ ]:
cumpopoffset = feb.sort_values("population").population.shift(1, fill_value=0).cumsum()
plt.bar(cumpopoffset/1E6, feb.sort_values("population").percapitadeaths, width=feb.sort_values("population").population/1E6, align="edge")
plt.ylabel("per capita death rate")
plt.xlabel("US population affected (millions)")

In [ ]:
cumdeathoffset = feb.sort_values("deaths").population.shift(1, fill_value=0).cumsum()
plt.bar(cumdeathoffset/1E6, feb.sort_values("deaths").percapitadeaths, width=feb.sort_values("deaths").population/1E6, align="edge")
plt.ylabel("per capita death rate")
plt.xlabel("US population affected (millions)")

In [ ]:
cumpcoffset = feb.sort_values("percapitadeaths").population.shift(1, fill_value=0).cumsum()
plt.bar(cumpcoffset/1E6, feb.sort_values("percapitadeaths").percapitadeaths, width=feb.sort_values("percapitadeaths").population/1E6, align="edge")
plt.ylabel("per capita death rate")
plt.xlabel("US population affected (millions)")

In [ ]:
# And I show you this lovely S-shape as a warning, that when you see it, you 
# might remember that it represents random data that has been sorted, and that it
# represents lies.

In [ ]:
feb.head()

In [ ]:
(feb.deaths == 0 ).sum()

In [ ]:
(feb.deaths == 1 ).sum()